1. Importo mi modelo ganador

In [1]:
import numpy as np
import tensorflow as tf
print("Iniciando archivo...")
model = tf.keras.models.load_model("modelo_m2_fin.keras")
print("Modelo cargado correctamente.")
class_names = [str(i) for i in range(10)]

Iniciando archivo...
Modelo cargado correctamente.


2. Clasificación en tiempo real


1. Captura imágenes de la cámara en tiempo real usando OpenCV.
2. Convierte cada frame a escala de grises para simplificar el procesamiento.
3. Detecta bordes con Canny y los ensancha con dilatación para hacer más visibles los contornos.
4. Encuentra todos los contornos presentes en la imagen.
5. Filtra los contornos candidatos a ser un dígito usando tres criterios:
    Área razonable (evita objetos grandes como la cara).
    Relación de aspecto típica de un número.
    Número mínimo de vértices (evita contornos suaves como una cara).
6. Selecciona el contorno más grande entre los candidatos encontrados.
7. Recorta la región del dígito del frame original en escala de grises.
8. **Centra el dígito en un canvas cuadrado y lo redimensiona a 280×280 para que coincida con el tamaño esperado por el modelo.
9. Normaliza la imagen del dígito** dividiendo los pixeles entre 255.
10. Expande dimensiones para formar el batch requerido por el modelo.
11. Predice la clase del dígito usando la red neuronal.
12. Muestra en pantalla:
    El frame original con las probabilidades de cada clase.
    La imagen de entrada que realmente se está enviando al modelo.
13. Termina el programa cuando presionas la tecla q.

In [15]:
import cv2
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar bordes (muy robusto)
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digit_img = None

    candidates = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)

        # 1. Área razonable para un dígito (evita cara)
        if not (2000 < area < 90000):
            continue

        # 2. Relación de aspecto (evita cara)
        aspect = w / h
        if not (0.3 < aspect < 3.0):
            continue

        # 3. Evitar formas demasiado suaves (la cara tiene bordes suaves)
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
        if len(approx) < 6:  
            # las caras son suaves, los números tienen más bordes o irregularidades
            continue

        candidates.append((c, area))

    if len(candidates) > 0:
        # Agarrar el mejor candidato
        candidates.sort(key=lambda x: x[1], reverse=True)
        cnt = candidates[0][0]

        x, y, w, h = cv2.boundingRect(cnt)
        digit = gray[y:y+h, x:x+w]

        # Centrar en canvas cuadrado
        size = max(w, h)
        canvas = 255 * np.ones((size, size), dtype=np.uint8)
        x_offset = (size - w) // 2
        y_offset = (size - h) // 2
        canvas[y_offset:y_offset+h, x_offset:x_offset+w] = digit

        digit_img = cv2.resize(canvas, (280, 280))
    else:
        # Nada válido detectado
        digit_img = np.ones((280, 280), dtype=np.uint8) * 255

    # Preparar para predicción
    img_norm = digit_img.astype("float32") / 255.0
    img_exp = np.expand_dims(img_norm, axis=(0, -1))

    predictions = model.predict(img_exp)
    pred_probs = predictions[0]
    pred_class = np.argmax(pred_probs)

    # Mostrar probabilidades
    display_frame = frame.copy()
    for i, prob in enumerate(pred_probs):
        color = (0, 255, 0) if i == pred_class else (0, 0, 255)
        cv2.putText(display_frame, f"{class_names[i]}: {prob:.2f}",
                    (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Original + Probabilidades", display_frame)
    cv2.imshow("Entrada al modelo", cv2.cvtColor(digit_img, cv2.COLOR_GRAY2BGR))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms

3. OCR

1. Captura frames en tiempo real desde la cámara.
2. Convierte el frame a escala de grises y aplica desenfoque para reducir ruido.
3. Genera una imagen binarizada usando un umbral adaptativo que resalta los dígitos.
4. Diluye la imagen binaria para conectar trazos del dígito y facilitar la detección.
5. Encuentra todos los contornos presentes en la imagen procesada.
6. Filtra cada contorno según área, relación de aspecto y complejidad para descartar ruido o formas no válidas.
7. Recorta la región del posible dígito usando su bounding box.
8. Centra el recorte en un canvas cuadrado y lo redimensiona a 280×280, que es el tamaño requerido por el modelo.
9. Normaliza la imagen y la adapta a la forma necesaria para la predicción.
10. Predice el dígito y su probabilidad usando la red neuronal cargada.
11. Dibuja un recuadro de color indicando si la predicción es confiable y coloca la etiqueta con la clase y probabilidad.
12. Guarda hasta cuatro recortes de dígitos detectados para mostrarlos en miniatura en la parte inferior del frame.
13. Si no se detecta ningún candidato, muestra un mensaje indicándolo.
14. Muestra en pantalla dos ventanas:
    El frame original con bounding boxes, etiquetas y miniaturas.
    La imagen binarizada utilizada para encontrar contornos.
15. Finaliza cuando se presiona la tecla q.


In [14]:
import cv2
import numpy as np
import tensorflow as tf
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated = cv2.dilate(thresh, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    display = frame.copy()
    detected_any = False
    max_display_rois = 4
    roi_display_list = []

    for c in contours:
        x, y, cw, ch = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        if area < 1500 or area > 200000:
            continue
        aspect = cw / float(ch + 1e-6)
        if not (0.2 < aspect < 5.0):
            continue
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
        if len(approx) < 3:
            continue

        digit = gray[y:y+ch, x:x+cw]
        size = max(cw, ch)
        canvas = 255 * np.ones((size, size), dtype=np.uint8)
        xo = (size - cw) // 2
        yo = (size - ch) // 2
        try:
            canvas[yo:yo+ch, xo:xo+cw] = digit
        except Exception:
            continue

        roi_resized = cv2.resize(canvas, (280, 280))
        img_norm = roi_resized.astype("float32") / 255.0
        img_exp = np.expand_dims(img_norm, axis=(0, -1))

        try:
            preds = model.predict(img_exp, verbose=0)
        except Exception:
            continue

        probs = preds[0]
        pred_idx = int(np.argmax(probs))
        pred_prob = float(probs[pred_idx])

        color_box = (0, 255, 0) if pred_prob >= 0.60 else (0, 165, 255)
        cv2.rectangle(display, (x, y), (x + cw, y + ch), color_box, 2)

        label = f"{class_names[pred_idx]}:{pred_prob:.2f}"
        cv2.putText(display, label, (x, max(20, y - 6)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_box, 2)

        detected_any = True
        if len(roi_display_list) < max_display_rois:
            roi_display_list.append((roi_resized, label))

    if not detected_any:
        cv2.putText(display, "No se detectaron candidatos validos", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    for i, (roi_img, lbl) in enumerate(roi_display_list):
        dx = 10 + i * (140 + 10)
        dy = 10
        small = cv2.resize(roi_img, (140, 140))
        small_bgr = cv2.cvtColor(small, cv2.COLOR_GRAY2BGR)
        x_offset = dx
        y_offset = h - 160
        display[y_offset:y_offset+140, x_offset:x_offset+140] = small_bgr
        cv2.putText(display, lbl, (x_offset, y_offset + 155),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("OCR - Original con bounding boxes y probabilidades", display)
    cv2.imshow("Mascara binarizada", thresh)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


IA utilizada para código y redacción de descrpción. 